## Data PreProcessing

In [1]:
%pip install langchain spacy nltk pinecone-client python-dotenv

     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ------------ --------------------------- 0.3/1.0 MB 6.3 MB/s eta 0:00:01
     ---------------------- ----------------- 0.6/1.0 MB 5.9 MB/s eta 0:00:01
     ------------------------------- -------- 0.8/1.0 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------  1.0/1.0 MB 5.8 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
     - -------------------------------------- 0.5/14.9 MB 14.7 MB/s eta 0:00:01
     -- ------------------------------------- 1.1/14.9 MB 13.9 MB/s eta 0:00:01
     ---- ----------------------------------- 1.7/14.9 MB 13.4 MB/s eta 0:00:01
     ------ --------------------------------- 2.3/14.9 MB 13.5 MB/s eta 0:00:01
     ------- -------------------------------- 2.9/14.9 MB 13.4 MB/s eta 0:00:01
     --------- ------------------------------ 3.6/14.9 MB 13.4 MB/s

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [3]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pip in c:\users\mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages (23.0.1)
  Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
import pandas as pd
import re
from langchain.schema import Document
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [17]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mercy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mercy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
import pandas as pd
df_transcripts = pd.read_csv("Data/video_metadata_with_transcripts.csv")
df_transcripts.columns
df_transcripts.head()

,Number,Youtube_link,Subject,title,channel,description,length,publish_date,views,error,transcript
0,1,https://www.youtube.com/watch?v=tOaMRG8DX3U,An AI Agent that knows everything about your P...,An AI Agent that knows everything about your P...,ServiceNow Community,Supercharge Your Project Management with AI!\n...,738.0,20250221.0,1063.0,NaN,"Hey folks, how you doing Chris Thanky here? A..."
1,2,https://www.youtube.com/watch?v=vteLoWpNw8Q,What Is Agentic AI and Why Should I Care?,What Is Agentic AI and Why Should I Care?,ServiceNow Community,"In today’s solo-Lauren episode of Break Point,...",1158.0,20250124.0,1936.0,NaN,In today's Solo Lauren episode of the Breakpo...
2,3,https://www.youtube.com/watch?v=7WJ6lmxa1WQ,Agentic AI workflows for AIOps,Agentic AI workflows for AIOps,ServiceNow Community,"In this episode, we explore the transformative...",995.0,20250518.0,395.0,NaN,"Hello, everyone. Welcome to another episode o..."
3,4,https://www.youtube.com/watch?v=fqB-NcZmqXo,ServiceNow's agentic AI framework explained: W...,ServiceNow's agentic AI framework explained: W...,ServiceNow Community,"In this episode, we explore how ServiceNow’s A...",946.0,20250312.0,4048.0,NaN,"Hello everyone, welcome to another episode of..."
4,5,https://www.youtube.com/watch?v=ZYJqkxGrNiI,AI and Business Agility: Enhancing Human Intel...,AI and Business Agility: Enhancing Human Intel...,ServiceNow Community,In this episode of the ServiceNow Enterprise A...,730.0,20241022.0,60.0,NaN,Welcome everyone to another edition of the Se...


In [20]:
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r"\s+", " ", text)        # Remove extra whitespace
    text = re.sub(r"[^\w\s]", "", text)     # Remove punctuation
    return text.lower().strip()

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized)

def merge_metadata_transcripts(metadata_path, transcript_path):
    df_meta = pd.read_csv(metadata_path)
    df_trans = pd.read_csv(transcript_path)
     # Merge on 'Youtube_link' (or rename to match)
    df = pd.merge(df_meta, df_trans, on="Youtube_link", how="inner")
    df["cleaned_transcript"] = df["Transcript"].apply(clean_text).apply(tokenize_and_lemmatize)
    return df

def prepare_langchain_docs(df):
    docs = []
    for _, row in df.iterrows():
        metadata = {
            "title": row["title"],
            "url": row["Youtube_link"],
            "subject": row["Subject"]
        }
        doc = Document(page_content=row["cleaned_transcript"], metadata=metadata)
        docs.append(doc)
    return docs

In [24]:
# File paths
metadata_file = "Data/ServiceNow_Youtube_Metadata_Clean.csv"
transcript_file = "Data/video_metadata_with_transcripts.csv"

# Step 1: Merge and preprocess
df_clean = merge_metadata_transcripts(metadata_file, transcript_file)

# Step 2: Convert to LangChain docs
langchain_docs = prepare_langchain_docs(df_clean)

# Optional: Print sample
print(langchain_docs[0])

KeyError: 'Transcript'

In [ ]:
Document(
    page_content="this is lemmatized transcript text",
    metadata={'title': 'AI and Business Agility', 'url': 'https://...', 'subject': 'AI for Business'}
)

In [6]:
# Save processed data
output_file = "Data/processed_transcripts.csv"
try:
    df_processed = pd.DataFrame(processed_data)
    df_processed.to_csv(output_file, index=False, encoding='utf-8')
    logging.info(f"Saved {len(df_processed)} transcript chunks to {output_file}")
except Exception as e:
    logging.error(f"Error saving processed data: {e}")
    raise

# Log missing transcripts
if missing_transcripts:
    logging.warning(f"Missing transcripts for video IDs: {missing_transcripts}")

# Summary
print(f"Processed {len(df_processed)} transcript chunks to {output_file}")
if missing_transcripts:
    print(f"Missing transcripts for {len(missing_transcripts)} videos: {missing_transcripts}")

2025-06-25 18:50:53,834 - INFO - Saved 327 transcript chunks to Data/processed_transcripts.csv


Processed 327 transcript chunks to Data/processed_transcripts.csv


In [8]:
df_processed = pd.read_csv("data/processed_transcripts.csv")
print(df_processed.head())

   video_id chunk_id                                               text  \
0         1      1_0  hey folk Chris Thanky like welcome session pla...   
1         1      1_1  info customer comment say project manager come...   
2         1      1_2  like practice give example mention customer wa...   
3         1      1_3  send information interpret aggregate pull know...   
4         1      1_4  answer go auto button LLM start tell look tell...   

                                             subject  
0  An AI Agent that knows everything about your P...  
1  An AI Agent that knows everything about your P...  
2  An AI Agent that knows everything about your P...  
3  An AI Agent that knows everything about your P...  
4  An AI Agent that knows everything about your P...  


In [11]:
missing = df[df["transcript"].isna()]["Number"].tolist()
print(f"Missing transcripts for video IDs: {missing}")

Missing transcripts for video IDs: []


In [13]:
df_metadata = pd.read_csv("data/ServiceNow_Youtube_Metadata_Clean.csv")
df_transcripts = pd.read_csv("data/video_metadata_with_transcripts.csv")
df_processed = pd.read_csv("data/processed_transcripts.csv")

# Validation checks
metadata_success = df_metadata["title"].notnull().sum()
missing_transcripts = df_transcripts["transcript"].isna().sum()
total_chunks = len(df_processed)
sample_chunk = df_processed["text"].iloc[0] if not df_processed.empty else "No chunks"

import os

# Ensure the 'docs' directory exists
os.makedirs("docs", exist_ok=True)

# Save report
with open("docs/validation_report.txt", "w") as f:
    f.write(f"Metadata: {len(df_metadata)} videos, {metadata_success} with valid metadata\n")
    f.write(f"Transcripts: {len(df_transcripts)} videos, {missing_transcripts} missing\n")
    f.write(f"Processed Chunks: {total_chunks}\n")
    f.write(f"Sample Chunk:\n{sample_chunk}\n")
print("Validation report saved to docs/validation_report.txt")

# Print summary
print(f"Metadata success: {metadata_success}/{len(df_metadata)}")
print(f"Missing transcripts: {missing_transcripts}/{len(df_transcripts)}")
print(f"Processed chunks: {total_chunks}")

Validation report saved to docs/validation_report.txt
Metadata success: 21/22
Missing transcripts: 0/22
Processed chunks: 327


In [15]:
mkdir src